## TESTE ANN

Import dependencies

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

Define to get all features (id excluded) ...

In [2]:
def preprocess_x(df_x):
    return df_x[['x0','y0','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5','x6','y6','x7','y7','x8','y8','x9','y9']].values

Define to get all results (id excluded)...

In [3]:
def preprocess_y(df_y):
    return df_y[["slope", "intercept"]].values

Load train datasets, process collumns and split in train/test data 90%/10%

In [4]:
x_data = pd.read_csv("train_100k.csv")
x_data = preprocess_x(x_data)

y_data = pd.read_csv("train_100k.truth.csv")
y_data =  preprocess_y(y_data)

x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(x_data,
                                                                        y_data,
                                                                        test_size = 0.1,
                                                                        random_state = 100)

Load test datasets and process collumns

In [5]:
x_data_eval = pd.read_csv("train_100k.csv")
x_data_eval = preprocess_x(x_data_eval)

## The ANN
The first, I'm tring to figure out how transfor the two output in one point. So i read again the problem set and decide only to make a simple artificial neural network with two output in a linear activation function (no activation at all).

### ANN Configuration  
Some tests:  
Epoch: 1000 cost = 0.138191439 cost2 = 47.019507828 4x200 learning rate 0.001   
Epoch: 1000 cost = 0.017634519 cost2 = 11.450007486 5x400 learning rate 0.001  
Epoch: 1000 cost = 0.008747602 cost2 = 6.681912537 4x500 learning rate 0.0001  

So, how more hidden reurons and smaller learning rate, more performance in the results...

In [6]:
learning_rate = 0.0001
training_epochs = 5000
batch_size = 1000
display_step = 100

train_data_size = 90000
test_data_size = 10000
data_size = train_data_size + test_data_size

n_hidden = 1000 # number of neurons on hidden layer

n_input = 20
n_output = 2

Create features and response tensors

In [7]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_output])
dropout_keep_prob = tf.placeholder(tf.float32)

Create tensors w'll be optimized:  
5 hidden layers  
1 output layer  
5 hidden bias  
1 output bias  

In [8]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden],stddev=0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h3': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h4': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'h5': tf.Variable(tf.random_normal([n_hidden, n_hidden],stddev=0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden, n_output],stddev=0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden])),
    'b2': tf.Variable(tf.random_normal([n_hidden])),
    'b3': tf.Variable(tf.random_normal([n_hidden])),
    'b4': tf.Variable(tf.random_normal([n_hidden])),
    'b5': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_output]))
}

Make a fully connected ANN ..

In [9]:
def make_network(x, weights, biases, dropout_keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.dropout(tf.nn.tanh(layer_1), dropout_keep_prob)

    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.dropout(tf.nn.tanh(layer_2), dropout_keep_prob)

    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.dropout(tf.nn.tanh(layer_3), dropout_keep_prob)
    
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.dropout(tf.nn.tanh(layer_4), dropout_keep_prob)
    
    layer_5 = tf.add(tf.matmul(layer_4, weights['h5']), biases['b5'])
    layer_5 = tf.nn.tanh(layer_5)

    out_layer = tf.matmul(layer_5, weights['out']) + biases['out']
    return out_layer

Run feed forward ...

In [10]:
y_pred = make_network(x, weights, biases, dropout_keep_prob)

Calculate MSE

In [11]:
slope_cost = tf.reduce_mean(tf.square(y_pred[:,0]-y[:,0]))

Calculate MAE

In [12]:
intercept_cost = tf.reduce_mean(tf.abs(y_pred[:,1]-y[:,1]))

Run backward propagation to adjust cost

In [13]:
slope_optimizer =  tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(slope_cost)
intercept_optimizer =  tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(intercept_cost)

Create batches to train in more small steps an

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(train_data_size / batch_size)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        avg_cost2 = 0
        
        for i in range(total_batch):
            batch_x, batch_y = x_data_train[i * batch_size:min(i * batch_size + batch_size, train_data_size), :], \
                             y_data_train[i * batch_size:min(i * batch_size + batch_size, train_data_size), :]
            # Train batch
            sess.run([slope_optimizer,  intercept_optimizer], feed_dict={x: batch_x, y: batch_y, dropout_keep_prob: 0.9})
            # Calculate cost
            slope_c, intercept_c = sess.run([slope_cost, intercept_cost], feed_dict={x: batch_x, y: batch_y, dropout_keep_prob: 1.})
            # Calculate avg cost
            avg_cost += slope_c / total_batch
            avg_cost2 += intercept_c / total_batch
        
        if (epoch+1) % display_step == 0:
            print('Epoch:', (epoch + 1), 'Slope MSE cost=', '{:.9f}'.format(avg_cost), 'Intercept MAE cost=', '{:.9f}'.format(avg_cost2))
    
    # Generate train result 
    r = sess.run(y_pred, feed_dict = {x: x_data, dropout_keep_prob: 1.})
    r = pd.DataFrame(r, columns=["slope", "intercept"])
    r.to_csv("submission.train_100k.csv", index=True, index_label='id')
    
    # Generate test result
    r = sess.run(y_pred, feed_dict = {x: x_data_eval, dropout_keep_prob: 1.})
    r = pd.DataFrame(r, columns=["slope", "intercept"])
    r.to_csv("submission.test_100k.csv", index=True, index_label='id')

Epoch: 100 Slope MSE cost= 0.092724972 Intercept MAE cost= 20.650480058
Epoch: 200 Slope MSE cost= 0.029275044 Intercept MAE cost= 14.340603648
Epoch: 300 Slope MSE cost= 0.015431797 Intercept MAE cost= 11.042841201
Epoch: 400 Slope MSE cost= 0.010297216 Intercept MAE cost= 8.920403025
Epoch: 500 Slope MSE cost= 0.007844613 Intercept MAE cost= 7.408725855
Epoch: 600 Slope MSE cost= 0.006090730 Intercept MAE cost= 6.450064129
Epoch: 700 Slope MSE cost= 0.005266885 Intercept MAE cost= 5.814264758
Epoch: 800 Slope MSE cost= 0.004933458 Intercept MAE cost= 5.276647393
Epoch: 900 Slope MSE cost= 0.003926165 Intercept MAE cost= 4.914394490
Epoch: 1000 Slope MSE cost= 0.003533717 Intercept MAE cost= 4.538295767
Epoch: 1100 Slope MSE cost= 0.003262276 Intercept MAE cost= 4.662337091
Epoch: 1200 Slope MSE cost= 0.002920492 Intercept MAE cost= 4.189010008
Epoch: 1300 Slope MSE cost= 0.003018943 Intercept MAE cost= 4.062330151
Epoch: 1400 Slope MSE cost= 0.002576584 Intercept MAE cost= 3.91028707

```bash
$ python evaluate.py train_100k.truth.csv submission.train_100k.csv  
Slope mse: 0.00446848524925  
Intercept mae: 3.44408998703  
```